# Smart Idiom Translator

In [108]:
##!pip install openai faiss-cpu
#pip install -qU langchain langchain-community
#!pip install -qU langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.5/443.5 kB 11.7 MB/s eta 0:00:00


In [48]:
import openai
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.vectorstores import FAISS
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import create_history_aware_retriever
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.output_parsers import StrOutputParser

import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import gradio as gr
import re
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
import pandas as pd
import gradio as gr
import faiss
import numpy as np
from openai import OpenAI
from langchain.schema import Document

from google.colab import userdata
key = userdata.get('OPENAI_API_KEY')
import os
os.environ['OPENAI_API_KEY'] = key


In [137]:
# Load idioms
df = pd.read_excel("/content/Idioms.xlsx")

In [166]:
# Create embeddings =====
# embed data sources
def embed(data, model):
  model_kwargs = {'device': "cpu"}
  encode_kwargs = {'normalize_embeddings': False}

  embeddings = HuggingFaceEmbeddings(
    model_name = model,
    model_kwargs = model_kwargs,
    encode_kwargs = encode_kwargs
  )
  return embeddings

# Prepare data for FAISS
df["context"] = df["Phrase"].astype(str) + " ||| LITERAL: " + df["Literal Translation"].astype(str) + " ||| ACTUAL: " + df["Actual Meaning"].astype(str)
context = df["context"].tolist()

def store_data(data, embeddings):
  # vector store
  # Create Document objects from the context list
  documents = [Document(page_content=text) for text in data]
  db = FAISS.from_documents(documents, embeddings)
  return db

embeddings = embed(context, 'sentence-transformers/all-MiniLM-L6-v2')
db = store_data(context, embeddings)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [167]:
llm = ChatOpenAI(model = "gpt-4o")

# Define the chat prompt
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a smart language translator for Igbo language, especially idioms. You can identify if there is any idiom(s) in a user's input sentence and produce two outputs: a concise literal translation and a concise figurative (actual) meaning."),
         MessagesPlaceholder("chat_history"),
        ("human", "Use only the retrieved {context} to provide the translation to the user sentence input {input}.")
    ]
)

# Create RAG chain

contextualize_q_system_prompt = """Given an Igbo sentence, \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
### Statefully manage chat history ###

messages_history = {}

def get_session_history(session_id: str):
    if session_id not in messages_history:
        messages_history[session_id] = ChatMessageHistory()
    return messages_history[session_id]



In [168]:
retriever=db.as_retriever(search_type="similarity", search_kwargs={"k": 5})

history_aware_retriever = create_history_aware_retriever(llm, retriever, contextualize_q_prompt)

question_answer_chain = create_stuff_documents_chain(llm, prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)


# Response Generation
def generate_response(query):

    return conversational_rag_chain.invoke({"input": query}, config={"configurable": {"session_id": "1"}})["answer"]


In [176]:
def get_meaning(user_input, meaning_type):
    response = generate_response(user_input)

    literal_match = re.search(r"LITERAL:\s*([^|]+)", response)
    actual_match = re.search(r"ACTUAL:\s*(.*)", response)

    literal = literal_match.group(1) if literal_match else None
    actual = actual_match.group(1) if actual_match else None

    if meaning_type.lower() == "literal":
        return literal
    else:
        return actual


In [180]:
#  Gradio UI
with gr.Blocks() as app:
    gr.Markdown("## 🗣 Igbo Idiom Translator")
    gr.Markdown("Enter an Igbo sentence and select whether to see its Literal or Figurative meaning.")

    with gr.Row():
        user_input = gr.Textbox(
            label="Enter Igbo sentence",
            placeholder="Type an Igbo idiom here..."
        )

    meaning_choice = gr.Radio(
        ["Literal", "Actual"],
        label="Meaning Type",
        value="Literal"
    )

    translate_btn = gr.Button("Translate")
    output_text = gr.Textbox(label="Meaning", interactive=False)

    translate_btn.click(
        fn=get_meaning,
        inputs=[user_input, meaning_choice],
        outputs=output_text
    )

app.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6f73c14d9fd9f57c1f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
